In [1]:
import numpy as np
import scipy as sp
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
url = "https://raw.githubusercontent.com/pedroteche-ih/DAFT_MEX_202209/main/data/nypd_murder_locations.csv"
tb_murder = pd.read_csv(url, parse_dates=['CMPLNT_FR_DT'])

In [5]:
tb_murder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CMPLNT_NUM           261 non-null    int64         
 1   CMPLNT_FR_DT         261 non-null    datetime64[ns]
 2   LOC_OF_OCCUR_DESC    261 non-null    object        
 3   BORO_NM              261 non-null    object        
 4   ADDR_PCT_CD          261 non-null    float64       
 5   LOC_OF_OCCUR_DESC.1  261 non-null    object        
 6   LAT                  261 non-null    float64       
 7   LONG                 261 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 16.4+ KB
